# Multiple outputs example
This notebook is a simple example of how to make a component with multiple outputs using the Pipelines SDK.

## Before running notebook:

### Setup notebook server
This pipeline requires you to [setup a notebook server](https://www.kubeflow.org/docs/notebooks/setup/) in the Kubeflow UI.  After you are setup, *upload the notebook in the Kubeflow UI* and then run it in the notebook server.

### Create a GCS bucket
This pipeline requires a GCS bucket.  If you haven't already, [create a GCS bucket](https://cloud.google.com/storage/docs/creating-buckets) to run the notebook.  Make sure to create the storage bucket in the same project that you are running Kubeflow on to have the proper permissions by default.  You can also create a GCS bucket by running `gsutil mb -p <project_name> gs://<bucket_name>`.

### Upload the notebook in the Kubeflow UI
In order to run this pipeline, make sure to upload the notebook to your notebook server in the Kubeflow UI.  You can clone this repo in the Jupyter notebook server by connecting to the notebook server and then selecting New > Terminal.  In the terminal type `git clone https://github.com/kubeflow/pipelines.git`.

### Install Kubeflow pipelines
Install the `kfp` package if you haven't already.

In [ ]:
!python3 -m pip install 'kfp>=2.0.0' --quiet

## Setup project info and imports

In [ ]:
from kfp import client, dsl
from typing import NamedTuple

## Create component
In order to create a component with multiple outputs, use `NamedTuple` with the same syntax as below.

In [ ]:
@dsl.component()
def product_sum(a: float, b: float) -> NamedTuple(
        'output', [('product', float), ('sum', float)]):
    '''Returns the product and sum of two numbers'''
    from collections import namedtuple
    
    product_sum_output = namedtuple('output', ['product', 'sum'])
    return product_sum_output(a*b, a+b)

## Create and run pipeline
### Create pipeline
The pipeline parameters are specified in the `pipeline` function signature.

In [ ]:
@dsl.pipeline(
    name='multiple-outputs-pipeline',
    description='Sample pipeline to showcase multiple outputs'
)
def pipeline(a: float=2.0, b: float=2.5, c: float=3.0):
    prod_sum_task = product_sum(a=a, b=b)
    prod_sum_task2 = product_sum(a=b, b=c)
    prod_sum_task3 = product_sum(a=prod_sum_task.outputs['product'],
                                 b=prod_sum_task2.outputs['sum'])

### Run pipeline

In [ ]:
arguments = {
    'a': 2.0,
    'b': 2.5,
    'c': 3.0,
}

kfp_endpoint = None
kfp_client = client.Client(host=kfp_endpoint)
run = kfp_client.create_run_from_pipeline_func(pipeline, arguments={})